In [1]:
!pip install transformers

     |████████████████████████████████| 778kB 2.7MB/s 
     |████████████████████████████████| 3.0MB 11.3MB/s 
     |████████████████████████████████| 890kB 28.4MB/s 
     |████████████████████████████████| 1.1MB 34.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=99f00ae5188ca5f77f1aacf82cfb0cde02a4a069e26aea34ba6b730e0b7c0e10
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
!pip install seqeval

  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=358cac6e3d3754556ca48154b8dedd953f318b1bb5fcfde107250270780e5d64
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [3]:
pip install tensorboardx

     |████████████████████████████████| 317kB 2.8MB/s 


In [4]:
!pip install simpletransformers

     |████████████████████████████████| 215kB 2.6MB/s 
     |████████████████████████████████| 7.2MB 7.2MB/s 
     |████████████████████████████████| 71kB 8.1MB/s 
     |████████████████████████████████| 1.4MB 37.8MB/s 
     |████████████████████████████████| 4.4MB 40.5MB/s 
     |████████████████████████████████| 102kB 9.8MB/s 
     |████████████████████████████████| 112kB 36.9MB/s 
     |████████████████████████████████| 102kB 7.8MB/s 
     |████████████████████████████████| 163kB 36.5MB/s 
     |████████████████████████████████| 122kB 45.4MB/s 
     |████████████████████████████████| 122kB 43.5MB/s 
     |████████████████████████████████| 71kB 9.0MB/s 
     |████████████████████████████████| 71kB 8.3MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.3-cp36-none-any.whl size=73870 sha256=15363562ee9038d86db140817e13649e507637b7278c57e1fa6f7c4e29f60009
  Stored in directory: /root/.cache/pip/wheels/a8/1d/38/2c19bb311f67cc7b4d07a2ec5ea36ab1a0a0ea50db994a5bc7
  Created wheel for

In [5]:
import pandas as pd

In [6]:
!unzip -qq '/content/drive/My Drive/Analytics Vidhya/input/train.zip'

In [7]:
train_df = pd.read_csv('/content/train.csv')

In [8]:
train_df.head()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0


In [9]:
train_df['text'] = train_df['TITLE'] + train_df['ABSTRACT']

In [10]:
!unzip -qq '/content/drive/My Drive/Analytics Vidhya/input/test.zip'

In [11]:
test_df = pd.read_csv('/content/test.csv')

In [12]:
test_id = test_df['ID']

In [13]:
test_df['text'] = test_df['TITLE'] + test_df['ABSTRACT']

In [14]:
train_df['labels'] = list(zip(train_df['Computer Science'].tolist(), train_df['Physics'].tolist(), train_df['Mathematics'].tolist(), train_df['Statistics'].tolist(),  train_df['Quantitative Biology'].tolist(), train_df['Quantitative Finance'].tolist()))
train_df['text'] = train_df['text'].apply(lambda x: x.replace('\n', ' '))

train_df.head()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,text,labels
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0,Reconstructing Subject-Specific Effect Maps P...,"(1, 0, 0, 0, 0, 0)"
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0,Rotation Invariance Neural Network Rotation i...,"(1, 0, 0, 0, 0, 0)"
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0,Spherical polyharmonics and Poisson kernels fo...,"(0, 0, 1, 0, 0, 0)"
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0,A finite element approximation for the stochas...,"(0, 0, 1, 0, 0, 0)"
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0,Comparative study of Discrete Wavelet Transfor...,"(1, 0, 0, 1, 0, 0)"


In [15]:
from simpletransformers.classification import MultiLabelClassificationModel


model = MultiLabelClassificationModel('roberta', 'roberta-base', num_labels=6, args={'train_batch_size':32, 'gradient_accumulation_steps':16, 'learning_rate': 3e-5, 'num_train_epochs': 6, 'max_seq_length': 256})

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultiLabelSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'clas

In [16]:
model.train_model(train_df)

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


In [18]:
to_predict = test_df.text.apply(lambda x: x.replace('\n', ' ')).tolist()
preds, outputs = model.predict(to_predict)

In [19]:
preds[0]

[1, 0, 0, 1, 0, 0]

In [20]:
outputs[0]

array([0.64342475, 0.01278362, 0.13953379, 0.8946625 , 0.01005486,
       0.00902291], dtype=float32)

In [21]:
outputs.shape

(8989, 6)

In [22]:
label_cols = ['Computer Science', 'Physics' , 'Mathematics' , 'Statistics' , 'Quantitative Biology' , 'Quantitative Finance']

In [23]:
submission = pd.DataFrame({'ID': test_id})
submission = pd.concat([submission, pd.DataFrame(outputs, columns = label_cols)], axis=1)

submission.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0.643425,0.012784,0.139534,0.894662,0.010055,0.009023
1,20974,0.019856,0.981392,0.018401,0.019651,0.017201,0.016550
2,20975,0.964769,0.018593,0.170129,0.032910,0.009751,0.010245
3,20976,0.021408,0.980000,0.015472,0.021214,0.021240,0.015587
4,20977,0.973560,0.034279,0.090599,0.022639,0.012181,0.010393


In [24]:
submission.to_csv('/content/drive/My Drive/Analytics Vidhya/output/sub3_proba.csv', index=False)

**Threshold 0.2**

In [25]:
sub3 = submission.copy()

In [26]:
for col in label_cols:
  sub3[col] = sub3[col].apply(lambda x:1 if x>0.2 else 0)

In [27]:
sub3.to_csv('/content/drive/My Drive/Analytics Vidhya/output/sub3_0.2.csv', index=False)

**Threshold 0.3**

In [28]:
sub3 = submission.copy()

In [29]:
for col in label_cols:
  sub3[col] = sub3[col].apply(lambda x:1 if x>0.3 else 0)

In [30]:
sub3.to_csv('/content/drive/My Drive/Analytics Vidhya/output/sub3_0.3.csv', index=False)

**Threshold 0.4**

In [31]:
sub3 = submission.copy()

In [32]:
for col in label_cols:
  sub3[col] = sub3[col].apply(lambda x:1 if x>0.4 else 0)

In [33]:
sub3.to_csv('/content/drive/My Drive/Analytics Vidhya/output/sub3_0.4.csv', index=False)

**Threshold 0.5**

In [34]:
sub3 = submission.copy()

In [35]:
for col in label_cols:
  sub3[col] = sub3[col].apply(lambda x:1 if x>0.5 else 0)

In [36]:
sub3.to_csv('/content/drive/My Drive/Analytics Vidhya/output/sub3_0.5.csv', index=False)

**Threshold 0.6**

In [37]:

sub3 = submission.copy()

In [38]:
for col in label_cols:
  sub3[col] = sub3[col].apply(lambda x:1 if x>0.6 else 0)

In [39]:
sub3.to_csv('/content/drive/My Drive/Analytics Vidhya/output/sub3_0.6.csv', index=False)

In [40]:
%%bash
cp -R '/content/outputs' '/content/drive/My Drive/Analytics Vidhya/models/roberta'